# CARGA LIBRERIAS

In [ ]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Lambda, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape, Conv2DTranspose,LeakyReLU, Embedding, Concatenate, GlobalMaxPooling2D, BatchNormalization
import matplotlib.pyplot as plt
#tf.random.set_seed(42)

In [ ]:
tf.test.gpu_device_name()
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

# CARGA MNIST

In [ ]:
# Carga
num_classes = 10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()


# Normalización
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_train2D = np.expand_dims(X_train, -1)
X_test2D = np.expand_dims(X_test, -1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(' -Entradas Entrenamiento:', X_train2D.shape)
print(' -Entradas Test:', X_test2D.shape)

print('Cambio de dimensiones para ajustar a la entrada de un Perceptrón:')
x_shape = X_train.shape
X_train = X_train.reshape(x_shape[0], x_shape[1]*x_shape[2])
X_test = X_test.reshape(len(X_test), x_shape[1]*x_shape[2])
print(' -Entradas Entrenamiento:', X_train.shape)
print(' -Entradas Test:', X_test.shape)

# MODELO AUTOENCODER PARA GENERAR ENCRIPTACIÓN

In [ ]:
# Tamaño de nuestra representación codificada
# 32 Factor de compresión = 24.5 (dado que la entrada es de tamaño 784)
# 16 Factor de compresión = 49 (dado que la entrada es de tamaño 784)
# con 16 el espacio latente es mas caotico

def AutoEncript(encoding_dim = 32):  

# Definimos las capas para la entrada, el encoder y el decoder:
# Capa de entrada
  input_img = Input(shape=(784,), name='Input')
  init = 'RandomNormal'

# Capas del encoder
  encoder1 = Dense(128, kernel_initializer=init, activation='relu', name='Encoder1')
  encoder2 = Dense(64, kernel_initializer=init,activation='relu', name='Encoder2')
  encoder3 = Dense(encoding_dim,kernel_initializer=init, activation='relu', name='Encoder3')

# Capas del decoder
  
  decoder1 = Dense(64,kernel_initializer=init, activation='relu', name='Decoder1')
  decoder2 = Dense(128,kernel_initializer=init, activation='relu', name='Decoder2')
  decoder3 = Dense(784,kernel_initializer=init, activation='sigmoid', name='Decoder3')

#       Autoencoder
# --------------------------
# "encoded" es la representación codificada de la entrada (cada vez más comprimida)
  encoded = encoder1(input_img)
  encoded = encoder2(encoded)
  encoded = encoder3(encoded)

# "decoded" es la reconstrución de la entrada a partir de la entrada codificada
  decoded = decoder1(encoded)
  decoded = decoder2(decoded)
  decoded = decoder3(decoded)

# Modelo que reconstruye una entrada
  autoencoder = tf.keras.Model(input_img, decoded)

# Creo encoder por separado
  encoder = tf.keras.Model(input_img, encoded)
# Creo decoder por separado
  input_img_e = Input(shape=(encoding_dim,), name='Input_e')
  decoded2 = decoder1(input_img_e)
  decoded2 = decoder2(decoded2)
  decoded2 = decoder3(decoded2)
  decoder = tf.keras.Model(input_img_e, decoded2)

  return autoencoder, encoder, decoder, input_img

autoencoder1, encoder1, decoder1, input_img1 = AutoEncript(16)
autoencoder2, encoder2, decoder2, input_img2 = AutoEncript(16)
# Visualizar arquitectura y dimensiones
print(f"{'*'*65}\n\t\t\tAutoencoder\n{'*'*65}")
autoencoder1.summary()
autoencoder2.summary()
encoder1.summary()
encoder2.summary()
decoder1.summary()
decoder2.summary()

In [ ]:
with tf.device('/device:GPU:0'):
  autoencoder1.compile(optimizer='adam', loss='binary_crossentropy')
  hist= autoencoder1.fit(X_train, X_train,
                      epochs=50,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(X_test, X_test))
  autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
  hist= autoencoder2.fit(X_train, X_train,
                      epochs=50,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(X_test, X_test))

# COMPROBACION ESPACIO LATENTE

- Se combinan los encoders y decoders para ver que los espacios latentes no son comunes para distintos modelos

In [ ]:
# Codificamos y decodificamos algunos dígitos de ejemplo (datos de test)
# -----------------------
encoded1_imgs = encoder1.predict(X_test)
encoded2_imgs = encoder2.predict(X_test)

decoded1_imgs = decoder1.predict(encoded1_imgs)
decoded2_imgs = decoder2.predict(encoded2_imgs)

decoded1_2_imgs = decoder1.predict(encoded2_imgs)
decoded2_1_imgs = decoder2.predict(encoded1_imgs)
 
def subplot(X,encoded_imgs, decoded_imgs, n_images = 10,encoding_dim = 32):
  # Entrada (original)
    plt.subplot(3, n_images, i + 1)
    plt.imshow(X[i].reshape(28, 28), cmap='gray')
    plt.title('Entrada')
    plt.axis('off')

    # Codificación
    plt.subplot(3, n_images, i + 1 + n_images)
    plt.imshow(encoded_imgs[i].reshape(1, encoding_dim), cmap='gray')
    #print(encoded_imgs[i].reshape(1, encoding_dim))
    plt.axis('off')
    plt.title('Entrada codificada')
    
    # Reconstrucción
    plt.subplot(3, n_images, i + 1 + n_images + n_images)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title('Reconstrucción')

def plot_data(X,encoded_imgs, decoded_imgs, n_images = 10,encoding_dim = 32):
  plt.figure(figsize=(30, 4))
  if n_images > 2:
    for i in range(n_images):
      subplot(X,encoded_imgs, decoded_imgs, n_images,encoding_dim)

plot_data(X_test,encoded1_imgs,decoded1_imgs,encoding_dim=16)